
<br>
==========<br>
Flood Fill<br>
==========<br>
Flood fill is an algorithm to identify and/or change adjacent values in an<br>
image based on their similarity to an initial seed point [1]_. The conceptual<br>
analogy is the 'paint bucket' tool in many graphic editors.<br>
.. [1] https://en.wikipedia.org/wiki/Flood_fill<br>
Basic example<br>
-------------<br>
First, a basic example where we will change a checkerboard square from white<br>
to mid-gray.<br>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, filters, color, morphology
from skimage.segmentation import flood, flood_fill

In [ ]:
checkers = data.checkerboard()

Fill a square near the middle with value 127, starting at index (76, 76)

In [ ]:
filled_checkers = flood_fill(checkers, (76, 76), 127)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))

In [ ]:
ax[0].imshow(checkers, cmap=plt.cm.gray)
ax[0].set_title('Original')

In [ ]:
ax[1].imshow(filled_checkers, cmap=plt.cm.gray)
ax[1].plot(76, 76, 'wo')  # seed point
ax[1].set_title('After flood fill')

In [ ]:
plt.show()

############################################################################<br>
Advanced example<br>
----------------<br>
<br>
Because standard flood filling requires the neighbors to be strictly equal,<br>
its use is limited on real-world images with color gradients and noise.<br>
The `tolerance` keyword argument widens the permitted range about the initial<br>
value, allowing use on real-world images.<br>
<br>
Here we will experiment a bit on the cameraman.  First, turning his coat from<br>
dark to light.

In [ ]:
cameraman = data.camera()

Change the cameraman's coat from dark to light (255).  The seed point is<br>
chosen as (155, 150)

In [ ]:
light_coat = flood_fill(cameraman, (155, 150), 255, tolerance=10)
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))

In [ ]:
ax[0].imshow(cameraman, cmap=plt.cm.gray)
ax[0].set_title('Original')
ax[0].axis('off')

In [ ]:
ax[1].imshow(light_coat, cmap=plt.cm.gray)
ax[1].plot(150, 155, 'ro')  # seed point
ax[1].set_title('After flood fill')
ax[1].axis('off')

In [ ]:
plt.show()

############################################################################<br>
The cameraman's  coat is in varying shades of gray. Only the parts of the coat<br>
matching the shade near the seed value is changed.<br>
<br>
Experimentation with tolerance<br>
------------------------------<br>
<br>
To get a better intuitive understanding of how the tolerance parameter<br>
works, here is a set of images progressively increasing the parameter with<br>
seed point in the upper left corner.

In [ ]:
output = []

In [ ]:
for i in range(8):
    tol = 5 + 20 * i
    output.append(flood_fill(cameraman, (0, 0), 255, tolerance=tol))

Initialize plot and place original image

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))
ax[0, 0].imshow(cameraman, cmap=plt.cm.gray)
ax[0, 0].set_title('Original')
ax[0, 0].axis('off')

Plot all eight different tolerances for comparison.

In [ ]:
for i in range(8):
    m, n = np.unravel_index(i + 1, (3, 3))
    ax[m, n].imshow(output[i], cmap=plt.cm.gray)
    ax[m, n].set_title('Tolerance {0}'.format(str(5 + 20 * i)))
    ax[m, n].axis('off')
    ax[m, n].plot(0, 0, 'bo')  # seed point

In [ ]:
fig.tight_layout()
plt.show()

############################################################################<br>
Flood as mask<br>
-------------<br>
<br>
A sister function, `flood`, is available which returns a mask identifying<br>
the flood rather than modifying the image itself.  This is useful for<br>
segmentation purposes and more advanced analysis pipelines.<br>
<br>
Here we segment the nose of a cat.  However, multi-channel images are not<br>
supported by flood[_fill].  Instead we Sobel filter the red channel to<br>
enhance edges, then flood the nose with a tolerance.

In [ ]:
cat = data.chelsea()
cat_sobel = filters.sobel(cat[..., 0])
cat_nose = flood(cat_sobel, (240, 265), tolerance=0.03)

In [ ]:
fig, ax = plt.subplots(nrows=3, figsize=(10, 20))

In [ ]:
ax[0].imshow(cat)
ax[0].set_title('Original')
ax[0].axis('off')

In [ ]:
ax[1].imshow(cat_sobel)
ax[1].set_title('Sobel filtered')
ax[1].axis('off')

In [ ]:
ax[2].imshow(cat)
ax[2].imshow(cat_nose, cmap=plt.cm.gray, alpha=0.3)
ax[2].plot(265, 240, 'wo')  # seed point
ax[2].set_title('Nose segmented with `flood`')
ax[2].axis('off')

In [ ]:
fig.tight_layout()
plt.show()

############################################################################<br>
Flood-fill in HSV space and mask post-processing<br>
------------------------------------------------<br>
<br>
Since flood fill operates on single-channel images, we transform here the<br>
image to the HSV (Hue Saturation Value) space in order to flood pixels of<br>
similar hue.<br>
<br>
In this example we also show that it is possible to post-process the binary<br>
mask returned by :func:`skimage.segmentation.flood` thanks to the functions<br>
of :mod:`skimage.morphology`.

In [ ]:
img = data.astronaut()
img_hsv = color.rgb2hsv(img)
img_hsv_copy = np.copy(img_hsv)

flood function returns a mask of flooded pixels

In [ ]:
mask = flood(img_hsv[..., 0], (313, 160), tolerance=0.016)
# Set pixels of mask to new value for hue channel
img_hsv[mask, 0] = 0.5
# Post-processing in order to improve the result
# Remove white pixels from flag, using saturation channel
mask_postprocessed = np.logical_and(mask,
                                    img_hsv_copy[..., 1] > 0.4)
# Remove thin structures with binary opening
mask_postprocessed = morphology.binary_opening(mask_postprocessed,
                                               np.ones((3, 3)))
# Fill small holes with binary closing
mask_postprocessed = morphology.binary_closing(
                mask_postprocessed, morphology.disk(20))
img_hsv_copy[mask_postprocessed, 0] = 0.5

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(color.hsv2rgb(img_hsv))
ax[0].axis('off')
ax[0].set_title('After flood fill')
ax[1].imshow(color.hsv2rgb(img_hsv_copy))
ax[1].axis('off')
ax[1].set_title('After flood fill and post-processing')

In [ ]:
fig.tight_layout()
plt.show()